In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams["font.family"] = "Arial"
from skimage import measure
import pyvista as pv
pv.set_jupyter_backend('trame')
from tqdm import tqdm, trange
from scipy import interpolate, ndimage, fft
from scipy.io import savemat
import numba as nb
from time import time

## Meshgrid

In [2]:
n_grid = 100
x = np.linspace(-1,1,n_grid+1)
y = np.linspace(-1,1,n_grid+1)
z = np.linspace(-1,1,n_grid+1)

r_grid = np.meshgrid(x,y,z) 


## Random wave

In [3]:
#### Wave vector distribution ####
def sample_k(k_mean,k_cov):
    return np.random.multivariate_normal(k_mean,k_cov)


In [4]:
#### Superpositioning the random wave ####
def sample_wave(r_grid,k_mean,k_cov,n_wave = 100):
    rho = np.zeros_like(r_grid[0])
    for i in range(n_wave):
        phi = np.random.rand()*2*np.pi # random phase
        k_sample = sample_k(k_mean,k_cov)
        k_dot_r = np.sum([r_grid[x]*k_sample[x] for x in range(3)],axis=0)
        rho_i = np.cos(k_dot_r + phi) # cos(k_n.r + phi_n)
        rho += rho_i

    rho = np.sqrt(2/n_wave)*rho
    
    return rho

def ball():
    rho = np.zeros_like(r_grid[0])
    radius = np.sqrt(r_grid[0]**2+r_grid[1]**2+r_grid[2]**2)
    rho[radius<=0.15]=1

    return rho

## Visualization

In [5]:
#### Wave vector distribution ####
kz_list = (np.arange(5)+1)*2
kxy_list = (np.arange(5)+1)*2
kz_grid, kxy_grid = np.meshgrid(kz_list,kxy_list)

n_wave = 60
alpha = 0 # clipping level

for k in zip(kz_grid.flatten(),kxy_grid.flatten()):
    print(k)
    k_mean = np.array([0,0,20])*np.pi # lamellar perpendicular to z axis 
    k_var  = (np.array([k[1],k[1],k[0]])*np.pi)**2
    k_cov  = np.diagflat(k_var)

    ### Clipping random wave ####
    rho = sample_wave(r_grid,k_mean,k_cov,n_wave = 60)

    # pyvista
    # https://stackoverflow.com/questions/6030098
    grid = pv.StructuredGrid(r_grid[0], r_grid[1], r_grid[2])
    grid["vol"] = rho.flatten('F')
    mesh = grid.contour([alpha])

    # Visualization
    pv.set_plot_theme('document')
    pl = pv.Plotter(window_size=[600, 600])
    pl.enable_anti_aliasing('msaa')

    backface_params = dict(color='#303030',
                        ambient=0.2, diffuse=0.8, specular=0.1, specular_power=10,
    #                        opacity=0.5
                        )
    pl.add_mesh(mesh, show_scalar_bar=False, color='#A0A0A0',  
                ambient=0.2, diffuse=0.8, specular=0.1, specular_power=10,
                backface_params=backface_params, 
                smooth_shading=True, 
    #             opacity=0.5
                )

    # camera setting
    pl.camera_position = 'yz'
    pl.camera.azimuth = -60.0
    pl.camera.elevation = 24.0
    pl.camera.reset_clipping_range()

    # light setting
    light = pv.Light()
    light.set_direction_angle(21, -55.0)
    light.attenuation_values = (0,0,2)
    pl.add_light(light)

    filename = './Figures/lamellar_{}-{}.png'.format(k[0],k[1])
    pl.show(screenshot=filename)